In [1]:
import os, sys
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")
import traci
import numpy as np
import pandas as pd
import gtsam
import gtsam_unstable

In [2]:
sumoBinary = "/usr/local/bin/sumo"
sumoCmd = [sumoBinary, "-c", "cross.sumocfg"]

seed = 1
randomState = np.random.RandomState(seed)

In [3]:
def printInfo():
    print(F"current time: {traci.simulation.getTime()}")
    print(F"current vehicle list: {traci.vehicle.getIDList()}")
    print(F"departed vehicle list: {traci.simulation.getDepartedIDList()}")
    print(F"arrived vehicle list: {traci.simulation.getArrivedIDList()}")
    ids = traci.vehicle.getIDList()
    for x in ids:
        pos = traci.vehicle.getPosition(x)
        angle = traci.vehicle.getAngle(x)
        print(F"{x}: x:{pos[0]}, y:{pos[1]}, angle:{angle / 180 * np.pi}")
    print("\n")

In [4]:
max_range = 20

gps_sigma_x, gps_sigma_y, gps_sigma_angle = 5, 5, np.pi * 20 / 180
odom_sigma_x, odom_sigma_y, odom_sigma_angle = gps_sigma_x / 10, gps_sigma_y / 10, gps_sigma_angle / 10
relative_sigma_x, relative_sigma_y, relative_sigma_angle = gps_sigma_x / 10, gps_sigma_y / 10, gps_sigma_angle / 10

gps_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array(
                        [gps_sigma_x, gps_sigma_y, gps_sigma_angle]))
odom_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array(
                        [odom_sigma_x, odom_sigma_y, odom_sigma_angle]))
relative_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array(
                        [relative_sigma_x, relative_sigma_y, relative_sigma_angle]))

In [5]:
lag = 10.0
smoother = gtsam_unstable.BatchFixedLagSmoother(lag)

new_factors = gtsam.NonlinearFactorGraph()
new_values = gtsam.Values()
new_timestamps = gtsam_unstable.FixedLagSmootherKeyTimestampMap()
name = {}

In [6]:
def symbol(x, t):
    return (name[x] << 15) + t

def pose(x):
    pos = traci.vehicle.getPosition(x)
    angle = traci.vehicle.getAngle(x) / 180 * np.pi
    return gtsam.Pose2(pos[0], pos[1], angle)

def noise_gps_measurement(x):
    current_pose = pose(x)
    noise_x = randomState.normal(0.0, gps_sigma_x)
    noise_y = randomState.normal(0.0, gps_sigma_y)
    noise_angle = randomState.normal(0.0, gps_sigma_angle)
    gps_measurement = current_pose.compose(gtsam.Pose2(
        noise_x, noise_y, noise_angle
    ))
    return gps_measurement

previous_states = {}
current_states = {}
def noise_odom_measurement(x):
    previous_pose = previous_states[x]
    current_pose = current_states[x]
    odom_measurement = previous_pose.between(current_pose)
    noise_x = randomState.normal(0.0, odom_sigma_x)
    noise_y = randomState.normal(0.0, odom_sigma_y)
    noise_angle = randomState.normal(0.0, odom_sigma_angle)
    odom_measurement = odom_measurement.compose(gtsam.Pose2(
        noise_x, noise_y, noise_angle
    ))
    return odom_measurement

def noise_relative_measurement(x, y):
    relative_measurement = pose(x).between(pose(y))
    noise_x = randomState.normal(0.0, relative_sigma_x)
    noise_y = randomState.normal(0.0, relative_sigma_y)
    noise_angle = randomState.normal(0.0, relative_sigma_angle)
    relative_measurement = relative_measurement.compose(gtsam.Pose2(
        noise_x, noise_y, noise_angle
    ))
    return relative_measurement

def euclidean_distance(x, y):
    pos_x = np.array(traci.vehicle.getPosition(x))
    pos_y = np.array(traci.vehicle.getPosition(y))
    return np.linalg.norm(pos_x - pos_y)

In [7]:
traci.start(sumoCmd)
data = []
time = 0
while time <= 63:
    print(F"time: {time}")
    if not traci.vehicle.getIDCount():
        traci.simulationStep()
        time += 1 
        continue
    printInfo()
    ids = traci.vehicle.getIDList()
    for x in ids:
        if x not in name:
            name[x] = len(name)

        current_key = symbol(x, time)
        new_timestamps.insert((current_key, time))

        # absolute measurement
        gps_measurement = noise_gps_measurement(x)
        new_factors.push_back(gtsam.PriorFactorPose2(
            current_key, gps_measurement, gps_noise
        ))
        new_values.insert(current_key, gps_measurement)
        
        current_states[x] = gps_measurement
        
        # temperal relative measurement
        if x in previous_states:
            previous_key = symbol(x, time - 1)
            odom_measurement = noise_odom_measurement(x)
            new_factors.push_back(gtsam.BetweenFactorPose2(
                previous_key, current_key, odom_measurement, odom_noise 
            ))

    # spatial relative measurement
#     for i in ids:
#         for j in ids:
#             if i == j or euclidean_distance(i, j) > max_range:
#                 continue
#             relative_measurement = noise_relative_measurement(i, j)
#             new_factors.push_back(gtsam.BetweenFactorPose2(
#                 symbol(i, time), symbol(j, time), relative_measurement, relative_noise
#             ))
            
    smoother.update(new_factors, new_values, new_timestamps)
    print("after optimization:")
    for x in ids:
        current_key = symbol(x, time)
        estimation = smoother.calculateEstimatePose2(current_key)
        groundtruth = pose(x)
        difference = groundtruth.between(estimation)
        print("Timestamp = " + str(time) + ", Vehicle = " + x)
        print(F"Estimation: {estimation}")
        print(F"GroundTruth: {groundtruth}")
        print(F"Difference: {groundtruth.x() - estimation.x(), groundtruth.y() - estimation.y(), difference.theta()}")
        value = {
            "timestamp": time, "vehicle": x, 
            "groundtruth_x":groundtruth.x(),
            "groundtruth_y":groundtruth.y(),
            "groundtruth_theta":groundtruth.theta(),
            "estimation_x":estimation.x(),
            "estimation_y":estimation.y(),
            "estimation_theta":estimation.theta(),
            "difference_x":groundtruth.x() - estimation.x(),
            "difference_y":groundtruth.y() - estimation.y(),
            "difference_theta":difference.theta(),
        }
        data.append(value)
    print('\n')

    print(F"end time {time}\n")

    new_timestamps.clear()
    new_values.clear()
    new_factors.resize(0)
    previous_states = current_states
    current_states = {}
            
    traci.simulationStep()
    time += 1 
df = pd.DataFrame(data)
df.to_csv("cooperative-localization-result.csv", index=False)
traci.close()

 Retrying in 1 seconds
time: 0
time: 1
time: 2
time: 3
current time: 3.0
current vehicle list: ('DU.0',)
departed vehicle list: ('DU.0',)
arrived vehicle list: ()
DU.0: x:92.0, y:194.9, angle:3.141592653589793


after optimization:
Timestamp = 3, Vehicle = DU.0
Estimation: (83.8783, 197.959, 2.95723)

GroundTruth: (92, 194.9, 3.14159)

Difference: (8.121726818316205, -3.058782068250366, -0.1843667218605023)


end time 3

time: 4
current time: 4.0
current vehicle list: ('DU.0',)
departed vehicle list: ()
arrived vehicle list: ()
DU.0: x:92.0, y:192.75685595781542, angle:3.141592653589793


after optimization:
Timestamp = 4, Vehicle = DU.0
Estimation: (97.1459, 188.793, 2.33694)

GroundTruth: (92, 192.757, 3.14159)

Difference: (-5.145925844957162, 3.9641324318155, -0.8046570921904579)


end time 4

time: 5
current time: 5.0
current vehicle list: ('DU.0',)
departed vehicle list: ()
arrived vehicle list: ()
DU.0: x:92.0, y:189.20650960281492, angle:3.141592653589793


after optimization:


after optimization:
Timestamp = 34, Vehicle = DU.3
Estimation: (88.3817, 55.6477, -2.6)

GroundTruth: (92, 49.5145, 3.14159)

Difference: (3.6183050206370524, -6.13321826579886, 0.5415967269376121)
Timestamp = 34, Vehicle = DU.4
Estimation: (86.8619, 95.2678, 2.85372)

GroundTruth: (92, 101.469, 3.14159)

Difference: (5.1381428785975345, 6.201423480693705, -0.2878682910603002)
Timestamp = 34, Vehicle = LR.0
Estimation: (86.6982, 99.9793, 1.82639)

GroundTruth: (85.399, 92, 1.5708)

Difference: (-1.299255929605735, -7.979288892544986, 0.2555892867430815)
Timestamp = 34, Vehicle = LR.1
Estimation: (83.9588, 98.129, 1.55425)

GroundTruth: (85.3988, 95.2, 1.5708)

Difference: (1.4400177087729134, -2.9289894141891324, -0.016550938144350904)
Timestamp = 34, Vehicle = RL.0
Estimation: (108.467, 107.961, -1.74267)

GroundTruth: (114.603, 108, -1.5708)

Difference: (6.1357597164009405, 0.039279390841286954, -0.17187387795084205)
Timestamp = 34, Vehicle = UD.0
Estimation: (101.198, 188.153, 0.44

In [8]:
df

,timestamp,vehicle,groundtruth_x,groundtruth_y,groundtruth_theta,estimation_x,estimation_y,estimation_theta,difference_x,difference_y,difference_theta
0,3,DU.0,92.000000,194.900000,3.141593,83.878273,197.958782,2.957226,8.121727,-3.058782,-0.184367
1,4,DU.0,92.000000,192.756856,3.141593,97.145926,188.792724,2.336936,-5.145926,3.964132,-0.804657
2,5,DU.0,92.000000,189.206510,3.141593,93.185475,182.147225,2.450031,-1.185475,7.059285,-0.691562
3,6,DU.0,92.000000,184.120272,3.141593,97.322752,185.061569,2.826837,-5.322752,-0.941297,-0.314755
4,6,DU.1,92.000000,194.900000,3.141593,86.276381,190.392046,-2.966189,5.723619,4.507954,0.175404
...,...,...,...,...,...,...,...,...,...,...,...
366,58,RL.0,13.540274,108.000000,-1.570796,9.593958,114.629422,-1.560336,3.946316,-6.629422,0.010461
367,59,LR.1,182.795250,95.200000,1.570796,184.322578,99.445753,1.456313,-1.527328,-4.245753,-0.114483
368,59,RL.0,5.173696,108.000000,-1.570796,5.567662,113.595614,-1.158945,-0.393966,-5.595614,0.411851
369,60,LR.1,190.744608,95.200000,1.570796,187.599002,98.406680,1.697276,3.145606,-3.206680,0.126480


In [9]:
error_x = np.sqrt(np.mean(df["difference_x"]**2))
error_y = np.sqrt(np.mean(df["difference_y"]**2))
error_theta = np.sqrt(np.mean(df["difference_theta"]**2))
print(error_x, error_y, error_theta / np.pi * 180)

5.04925180847583 4.915707834334122 20.243081059392686
